We will create a simple CNN that removes tiny random Gaussian noise from an MNIST image 

We begin by adding noise to an MNIST data set. Lets define a function that adds noise:

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

#The dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

#normalize
test_images = test_images/255.0
train_images = train_images/255.0
x_not  = test_images[20,:,:]
n=5000

In [2]:
def add_Gaussian_noise(Beta_schedule, samples):  
    noise = np.random.normal(0, Beta_schedule, size= (samples, 28,28))
    return noise


Now let us create the neural network. The idea is create a new loss function (MSE) between the actual noise and the predict noise by the Convo neural network. The archecitecture is as follows:  Input -> (28,28) image + noise sample -> conv2d 32 filter -> max pooling -> CNN 32 filter -> flatten -> Dense (1000) -> Dense (1000) -> Dense-> (28*28) -> Linear 

In [3]:
#We start by initializing the data
noisy_samples =add_Gaussian_noise(Beta_schedule=.4, samples=n)

In [21]:
#instantiating the model class in tensorflow
pred_model1 = models.Sequential()

pred_model1.add(tf.keras.layers.Conv2D(filters = 32, kernel_size =(3,3), padding = 'same', activation='relu',input_shape=(28, 28, 1))) # CNN 32 filter (1, 28,28, 32)

pred_model1.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2),
    padding='valid'
)) # max pooling

pred_model1.add(tf.keras.layers.Conv2D(filters = 32, kernel_size =(3,3), padding = 'same', activation='relu',input_shape=(28, 28, 1))) # second CNN 32 filter

pred_model1.add(tf.keras.layers.Flatten())  #flatten 

pred_model1.add(tf.keras.layers.Dense(784,activation='relu',use_bias=True)) # first dense layer with 784

pred_model1.add(tf.keras.layers.Dense(784,activation='relu',use_bias=True)) # second layer with 784 neurons

pred_model1.add(tf.keras.layers.Dense(784,activation='linear',use_bias=True)) # output  layer with 784 neurons

pred_model1.compile(optimizer='adam', loss= 'mean_squared_error', metrics=['mse', 'mae'])

C:\Users\ianau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


We need to make some small modification to our data.

In [5]:
X_train = np.zeros(noisy_samples.shape)
for i in range(n):
    X_train[i,:,:] = noisy_samples[i,:,:] + x_not

In [22]:
X_train_n = np.expand_dims(X_train,axis=-1)
print(X_train_n.shape)
noisy_samples_fin = noisy_samples.reshape((n,784))

type(noisy_samples_fin)


(5000, 28, 28, 1)


numpy.ndarray

Now to fit the model

In [24]:
pred_model1.fit(X_train_n, noisy_samples_fin, epochs=10)

Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.1603 - mae: 0.3196 - mse: 0.1603
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.1600 - mae: 0.3192 - mse: 0.1600
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.1602 - mae: 0.3194 - mse: 0.1602
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.1600 - mae: 0.3191 - mse: 0.1600
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.1600 - mae: 0.3191 - mse: 0.1600
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.1603 - mae: 0.3194 - mse: 0.1603
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.1602 - mae: 0.3193 - mse: 0.1602
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.1602 - mae: 0.3193 - mse: 0.1602
Epoch 9/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.1599 - mae: 0.3191 - mse: 0.1599
Epoch 10/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.1602 - mae: 0.3194 - mse: 0.1602


Test data

In [8]:
# number of evaluations
k=5

#We start by initializing the data
test_noisy_samples =add_Gaussian_noise(Beta_schedule=.2, samples=k)

#getting the noisy test images
test_noisy_images = np.zeros(test_noisy_samples.shape)
for i in range(k):
    test_noisy_images[i,:,:] = test_noisy_samples[i,:,:] + x_not

#getting the noise flattened
test_noisy_samples_fin = test_noisy_samples.reshape((k,784))

b = pred_model1.evaluate(test_noisy_images, test_noisy_samples_fin, verbose=2)



1/1 - 0s - 162ms/step - loss: 0.0372


In [38]:
#plt.imshow(test_noisy_images[1,:,:])
bae = test_noisy_images[1,:,:]


bae = np.expand_dims(bae, axis=0)

y_sam = X_train_n[0,:,:,0]
y_sam = np.expand_dims(y_sam,axis=-1)
y_sam = np.expand_dims(y_sam,axis=0)
fin = pred_model1.predict(y_sam)

#not learning
x = noisy_samples_fin[0,:] - fin
x = np.linalg.norm(x)
print(x)

fin = fin.reshape(1,28,28)
#print('This is the predicted noise',fin)


#print('This is the actual noise', test_noisy_samples[1,:,:])
hehe = test_noisy_samples_fin.reshape(k,28,28)
new = test_noisy_images[1,:,:] - fin #hehe[1,:,:]



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[[-1.24521237e-02 -3.64586245e-03 -2.90679163e-04  7.61101861e-03
   7.79514085e-05  1.19059356e-02  8.11603852e-03  6.47690427e-03
   7.40263704e-03 -1.92425447e-03 -4.27271379e-03  1.73127383e-03
   7.68159734e-05  4.92608128e-03 -9.65287443e-03  6.46115839e-03
  -2.68499629e-04  1.15468865e-02  8.06493033e-03  4.29763924e-03
  -3.72285466e-03  9.69692040e-03 -6.40993239e-04 -6.19068090e-03
   8.24871939e-03 -1.22332824e-02 -9.65425163e-04 -1.56556338e-03
  -2.20664917e-03 -2.13462184e-03  3.39970482e-03  3.19562200e-03
   1.42853486e-03  3.92972352e-03  6.12795725e-03  7.50674121e-03
   6.17214059e-03 -4.58936859e-03 -3.97714879e-03 -4.85036476e-03
   4.14098380e-03 -1.42482752e-02 -7.91058876e-04  1.77004852e-03
  -9.44697484e-03  1.05094118e-02 -6.98104908e-04 -1.41179319e-02
  -7.64303049e-03 -1.52422609e-02 -2.11915467e-03 -9.87972692e-03
  -5.84625266e-03  8.08097597e-04 -8.63517355e-03 -2.12736614e-03
   6.62623439e-03 -8.98163824e-04 -9.7